<a href="https://colab.research.google.com/github/JHyunjun/Attention/blob/main/AttentionLSTM_230308.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Created by Hyunjun,JANG
# limitsinx.tistory.com
# Last revision date : 2023.03.08

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

xy_train = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/train_v.txt',delimiter=',',dtype=np.float32) #5808
#xy_test = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/LSTM/test_v.txt',delimiter=',',dtype=np.float32) #10883
xy_test = xy_train

def MinMaxScaler(data) : # Normal Std
  numerator = data - np.min(data,0)
  denominator = np.max(data,0) - np.min(data,0)

  return numerator / (denominator + 1e-7)

xy_train = MinMaxScaler(xy_train)
xy_test = MinMaxScaler(xy_test)

x_train = xy_train[:,0:-1]
y_train = xy_train[:,[-1]]

'''
def origin_minmax(data) : #Mine
  arr_max = np.zeros(data.shape[1])
  arr_min = np.zeros(data.shape[1])
  print(arr_min)
  print(data.shape[1])
  for i in range(data.shape[1]) : 
    #print(i)
    max = np.max(data[:,i])
    min = np.min(data[:,i]) 
    arr_max[i] = max
    arr_min[i] = min
    pass
  return arr_max, arr_min

arr_max1, arr_min1 = origin_minmax(xy_train)
arr_max2, arr_min2 = origin_minmax(xy_test)

print(arr_max1, arr_min1)
print(arr_max2, arr_min2)

def MinMaxScaler(data):
  for i in range(data.shape[1]) : 
    max = np.max(data[:,i])
    min = np.min(data[:,i])
    data[:,i] = (data[:,i] - min) / (max - min + 1e-7)
    pass
  return data

def back_MinMax(data,max,min):
  print(max)
  print(min)
  for i in range(data.shape[1]) : 
    data[:,i] = data[:,i] * (max[i] - min[i]) + min[i]
    pass
  return data
'''

In [ ]:
# Data Sequence Setting

TIME_STEPS = 10
seq_length = TIME_STEPS

train_set = xy_train
test_set = xy_test

def build_dataset(time_series, seq_length) : 
  dataX = []
  dataY = []
  for i in range(0, len(time_series) -seq_length) : 
    x = time_series[i:i+seq_length , : -2]
    y = time_series[i + seq_length -1, [-1]]
    dataX.append(x)
    dataY.append(y)

  return np.array(dataX), np.array(dataY)

trainX, trainY = build_dataset(train_set, seq_length)
testX, testY = build_dataset(test_set, seq_length)

print(trainX.shape, trainY.shape) #(# of datas, Time_steps, features)

INPUT_DIM = trainX.shape[2]

In [ ]:
# Network Design

def attention_3d_block(inputs) : 
  input_dim = int(inputs.shape[2])
  a = tf.keras.layers.Permute((2,1))(inputs)
  a = tf.keras.layers.Dense(TIME_STEPS, activation = 'softmax')(a)
  a_probs = tf.keras.layers.Permute((2,1), name = 'attention_vec')(a)
  output_attention_mul = tf.keras.layers.multiply([inputs, a_probs])

  return output_attention_mul


def model_attention_applied_after_lstm() : 
  inputs = tf.keras.Input(shape = (TIME_STEPS, INPUT_DIM,))
  lstm_units = 64
  lstm_out = tf.keras.layers.LSTM(lstm_units, return_sequences = True)(inputs)
  attention_mul = attention_3d_block(lstm_out)
  attention_mul = tf.keras.layers.Flatten()(attention_mul)
  output = tf.keras.layers.Dense(1, activation = 'sigmoid')(attention_mul)
  model = tf.keras.Model(inputs = [inputs], outputs = output)

  return model

modelhj = model_attention_applied_after_lstm()
modelhj.compile(optimizer = 'adam', loss = 'mse')
modelhj.fit(trainX, trainY, epochs = 10, batch_size = 1, validation_split = 0)
  

In [ ]:
# Figure

trainY_predict = modelhj.predict(trainX)
plt.figure(figsize = (6,8))
plt.plot(trainY, color = 'blue')
plt.plot(trainY_predict, color = 'red')
plt.legend(['Real Voltage', 'Predicted Voltage'])

layer_outputs = [layer.output for layer in modelhj.layers if layer.name == 'attention_vec']
activation_model = tf.keras.models.Model(inputs = modelhj.input, outputs = layer_outputs)

attention_vectors = []

for i in range(100) : 
  predict_output = activation_model.predict(testX[i+1000,:,:].reshape(1,TIME_STEPS,trainX.shape[2]))
  dols = np.mean(predict_output, axis = 2).squeeze()
  assert(np.sum(dols) - 1.0) < 1e-5
  attention_vectors.append(dols)
  attention_vector_final = np.mean(np.array(attention_vectors),axis = 0)

pd.DataFrame(attention_vector_final, columns = ['attention(%)']).plot.bar()